# TP8 - Modelos de lenguaje y sistemas de recomendación

In [46]:
#librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

# Modelo de Lenguaje

## Funciones

In [47]:
# Funciones

# Retorna el texto dado, con los espacios unificados y en minúsculas
def processing_text (text):

    #Se separan las palabras por espacios
    words_list = text.split() 

    #Se concatenan las palabras agregando un único espacio
    text_unified = " ".join(words_list) 

    #Se pasan los caracteres a minuscula
    text_unified = text_unified.lower()

    return text_unified


# Retorna la similitud coseno entre los vectores u y v
def SC (u,v) :
    abs_u = np.linalg.norm(u)
    abs_v = np.linalg.norm(v)

    prod = u@v

    return prod/(abs_u * abs_v)

## Clases

In [48]:
# Clases

class semantic_search ():
        
    def __init__(self, model_name='sentence-transformers/paraphrase-multilingual-mpnet-base-v2'):

        #Modelo de lenguaje a usar
        self.model = SentenceTransformer(model_name)

        #Embeddings del texto
        self.embeddings = None
        self.texts = None

    def fit(self, text_list):
        #Se carga la lista de strings
        self.texts = text_list

        print('self.texts', len(self.texts))

        #Se computan los embeddings de la lista de strings dadas
        self.embeddings = self.model.encode(self.texts)

        print('self.embeddings', self.embeddings.shape)

    def search(self, query, k=5):

        #Se computa el embedding del texto buscado
        query_emb = self.model.encode(query)

        print('query_emb:',query_emb.shape)

        #Se calcula la similitud coseno entre cada embedding del texto almacenado y el del solicitado
        sims = np.array([SC(e, query_emb) for e in self.embeddings])

        print('sims', sims.shape)

        #Se ordenan de mayor a menor los primeros k indices de los embeddings similares
        idxs = np.argsort(sims)[::-1][:k]

        print(idxs.shape)

        #Se obtienen los primeros k textos y similitudes, ordenadas de mayor a menor
        ranked_texts = [self.texts[i] for i in idxs]
        ranked_sims = np.array([sims[i] for i in idxs])

        #Se retornan por separado los textos y las similitudes
        return ranked_texts, ranked_sims

## Procesamiento de datos

#### Carga de datos y preprocesamiento de los mismos

In [49]:
filename = 'anime.csv'
df = pd.read_csv(filename, sep=',', nrows=10)

print('df:', df.shape)

#Se obtienen los titulos
titles = df['title']

#Se procesan los titulos para eliminar espacios y pasar todo a minusculas
titles_processed = [processing_text(titles[i]) for i in range(df.shape[0])]

#Se vuelven a cargar los titulos en la columna de titulos del df
df['title'] = titles_processed

print('df:', df.shape)

df: (10, 1001)
df: (10, 1001)


#### Modelo de lenguaje

In [50]:
language_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# El modelo toma el texto, lo separa en tokens y reemplaza cada token por un ID entero del vocabulario que posee
# Tras esto, realiza un 'pooling', que puede ser un promedio, para dar el array final de 1x768
embeddig_att =language_model.encode(processing_text('Attack on Titan'))

#Se inicializa el modelo de búsqueda semántica
ss_model = semantic_search()

#Se entrena el modelo de búsqueda semántica, utilizando los títulos
ss_model.fit(df['title'].to_list())

#Se obtienen los 15 titulos más semejantes al query 'Giant' y su similitud coseno
texts,sims = ss_model.search(query = 'Giant', k=15)

# print('texts', len(texts))
# print('sims', sims.shape)

self.texts 10
self.embeddings (10, 768)
query_emb: (768,)
sims (10,)
(10,)


# Sistema de recomendación